**Run the following cell to download the data in advance**

In [1]:
import pooch
from pathlib import Path

voxel_sizes = [5, 2, 2] # in microns

data_path = Path.cwd().parent / "serial_2p_subset"
# Use pooch to fetch data if it hasn't already been downloaded
dsb_data_url = "https://gin.g-node.org/BrainGlobe/demo-materials/raw/master/serial2p_subset.zip"
data_path = pooch.retrieve(dsb_data_url, known_hash="c31e0136d00024ba74085baefbfd174c8e325f97a942c803ee720bdc8f6e7a00", progressbar=True, processor=pooch.Unzip(extract_dir=data_path))
data_path = Path(data_path[0]).parent.parent
signal_images = data_path / "ch00"
background_images = data_path / "ch02"

In [2]:
import glob
import os

from dask import array as da
from dask import delayed
from imlib.general.system import get_sorted_file_paths
from tifffile import TiffFile, imread

In [3]:
# Write a function to load data with dask
# Alternatively - `from cellfinder_core.tools.IO import read_with_dask`

lazy_imread = delayed(imread) 

def get_tiff_meta(path):
    with TiffFile(path) as tfile:
        nz = len(tfile.pages)
        if not nz:
            raise ValueError(f"tiff file {path} has no pages!")
        first_page = tfile.pages[0]

    return tfile.pages[0].shape, first_page.dtype


def read_with_dask(path):
    filenames = glob.glob(os.path.join(path, "*.tif"))
    shape, dtype = get_tiff_meta(filenames[0])
    lazy_arrays = [lazy_imread(fn) for fn in get_sorted_file_paths(filenames)]
    dask_arrays = [
        da.from_delayed(delayed_reader, shape=shape, dtype=dtype)
        for delayed_reader in lazy_arrays
    ]
    stack = da.stack(dask_arrays, axis=0)
    return stack

In [4]:
signal_array = read_with_dask(signal_images)
background_array = read_with_dask(background_images)

print(signal_array)
print(background_array)

dask.array<stack, shape=(100, 3868, 5416), dtype=uint16, chunksize=(1, 3868, 5416), chunktype=numpy.ndarray>
dask.array<stack, shape=(100, 3868, 5416), dtype=uint16, chunksize=(1, 3868, 5416), chunktype=numpy.ndarray>


In [5]:
from cellfinder_core.main import main as cellfinder_run
detected_cells = cellfinder_run(signal_array,background_array,voxel_sizes)

2023-06-01 15:26:36.306059: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 15:26:36.365230: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-01 15:26:36.680875: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-01 15:26:36.680907: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

Detection complete - all planes done in : 0:02:12.435058


2023-06-01 15:28:49.580821: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-01 15:28:49.600208: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


237/237 [==============================] - 117s 486ms/step


In [6]:
import napari
viewer = napari.view_image(background_array)
viewer.add_image(signal_array)

<Image layer 'signal_array' at 0x7feeb0573e50>

In [7]:
from cellfinder_napari.utils import cells_to_array
detected, rejected = cells_to_array(detected_cells)

viewer.add_points(
    rejected,
    name="Rejected",
    size=20,
    n_dimensional=True,
    opacity=0.8,
    symbol="ring",
    face_color="lightskyblue",
)
viewer.add_points(
    detected,
    name="Detected",
    size=20,
    n_dimensional=True,
    opacity=0.8,
    symbol="ring",
    face_color="lightgoldenrodyellow",
)

<Points layer 'Detected' at 0x7fec9c1fba60>